## HIV News articles extraction from Dhaka Tribune. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for HIV News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'HIV site:www.dhakatribune.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

242 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(article.meta_description.strip().replace(u'\xa0', u' ').replace('\r\n        \t \r\n        \r\n        \t',''))
            except:
                descriptions.append(soup.select_one('meta[name="description"]')['content'].replace('\r\n        \t&nbsp;\r\n        \r\n        \t', ''))
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(article.authors.strip())
            except:
                authors.append(soup.select_one('.ptt.author-bg').a.text.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(str(article.publish_date))
            except:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(article.text.strip())
            except:
                news.append(soup.select_one('div.report-content.fr-view').text.replace('\n', '').strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 16min 53sakatribune.com/feature/health-wellness/2017/09/22/sanofi-tests-three-one-antibody-treat-prevent-hiv/attachment/bigstock-196161664/1647834805237587830307623093_1953323341359665


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

242 242 242 242 242 242 242 242


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})
#tbl = tbl.dropna()
tbl.to_csv('Dhaka_Tribune.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries
0,Report: 193 tested HIV-positive in 8 southern ...,The report highlights that returning expatriat...,Tribune Desk,2018-09-30 00:00:00,Representational photo Bigstock\n\nThe report ...,"[report, hiv, 193, spreading, organization, kh...",Representational photo BigstockThe report high...
1,"HIV deaths: Bangladesh ranks 10th in Asia, Ind...","There are currently around 12,000 HIV-positive...",Tribune Desk,2018-01-26 00:00:00,Bangladesh recorded the 10th highest number of...,"[south, hiv, data, asia, ranks, asian, banglad...",Bangladesh recorded the 10th highest number of...
2,World AIDS Day observed: Tuberculosis patients...,The World Aids Day was observed yesterday acro...,"Hedait Hossain Molla, Khulna",2016-12-02 00:00:00,This year’s slogan for the day is “Let’s raise...,"[day, hiv, total, bangladesh, prevalence, viru...","According to the Mukta Akash Bangladesh, a non..."
3,"Fight HIV, not sex workers",We need to address the social problems that ma...,Md Abdul Quayyum,2016-12-18 00:00:00,“My father died in a road accident when I was ...,"[health, hiv, sex, fight, violence, bangladesh...",But sex workers are among those who are most v...
4,"Latest news update from HIV in Bangladesh, World",Latest HIV news in Bangladesh,None,None,9-year-old South African becomes third...\n\nA...,"[south, hiv, taken, old, given, bangladesh, th...",9-year-old South African becomes third...Afric...
